In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px


# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

<ipython-input-1-957ea83e9eed>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-1-957ea83e9eed>:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
d1 = spacex_df[[ 'Launch Site', 'class']]

In [3]:
df1 = d1.groupby('Launch Site').size().reset_index(name='class1')
df1

,Launch Site,class1
0,CCAFS LC-40,26
1,CCAFS SLC-40,7
2,KSC LC-39A,13
3,VAFB SLC-4E,10


In [4]:

df2 = spacex_df[spacex_df['Launch Site'] == 'CCAFS LC-40']
df2 = df2[[ 'Launch Site', 'class']]
df2 = df2.groupby('Launch Site').size().reset_index(name = 'class2')
df2

,Launch Site,class2
0,CCAFS LC-40,26


In [5]:
aa = (spacex_df['Payload Mass (kg)'] > 1000) & (spacex_df['Payload Mass (kg)'] < 3000)
spacex_df[aa]

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
7,7,9,CCAFS LC-40,0,2296.0,F9 v1.1,v1.1
8,8,10,CCAFS LC-40,0,1316.0,F9 v1.1,v1.1
11,11,13,CCAFS LC-40,0,2216.0,F9 v1.1 B1010,v1.1
12,12,14,CCAFS LC-40,0,2395.0,F9 v1.1 B1012,v1.1
15,15,17,CCAFS LC-40,0,1898.0,F9 v1.1 B1015,v1.1
17,17,19,CCAFS LC-40,1,1952.0,F9 v1.1 B1018,v1.1
18,18,20,CCAFS LC-40,1,2034.0,F9 FT B1019,FT
24,24,27,CCAFS LC-40,1,2257.0,F9 FT B1025.1,FT
33,33,49,VAFB SLC-4E,0,2150.0,F9 FT B1038.2,FT
36,36,30,KSC LC-39A,1,2490.0,F9 FT B1031.1,FT


In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site_dropdown', 
                                options=[ {'label': 'All Sites', 'value': 'ALL'}, 
                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'}, 
                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}, 
                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}], 
                                value='ALL', 
                                placeholder="Choose Launch Site", 
                                searchable=True ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload_slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                       0:'0', 2500:'2500', 5000:'5000', 7500:'7500', 10000:'10000'},
                                                value=[min_payload, max_payload]),
                                
                                

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site_dropdown` as input, `success-pie-chart` as output
@app.callback(
            Output('success-pie-chart', 'figure'),
            Input('site_dropdown', 'value'))
def get_pie_chart(entered_site):
        filtered_df = spacex_df
        if entered_site == 'ALL':
            df1 = filtered_df[['Launch Site', 'class']]
            df1 = df1.groupby('Launch Site').size().reset_index(name = 'class1')
            fig = px.pie(df1, values='class1', names='Launch Site', 
            title='all lauchsite')
            return fig
        else:
            df2 = spacex_df[spacex_df['Launch Site'] == entered_site]
            df2 = df2[[ 'Launch Site', 'class']]
            df2 = df2.groupby('class').size().reset_index(name = 'class2')

            fig2 = px.pie(df2, values='class2', names='class', title= f"success percentage of {entered_site}")
            return fig2
                                         
# TASK 4:
# Add a callback function for `site_dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output('success-payload-scatter-chart', 'figure'),
            Input('site_dropdown', 'value'),
            Input('payload_slider', 'value'))
def get_scatter_chart(site_dropdown, payload_slider):
    low, high= payload_slider
    mask = (spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)
    filteredDF = spacex_df[mask]
    if site_dropdown == 'ALL':
        fig = px.scatter(filteredDF, x='Payload Mass (kg)', y= 'class', color='Booster Version',
                        title='Payload and launchsite scatterpot')
        return fig
    else:
        df3 = filteredDF[filteredDF['Launch Site'] == site_dropdown]
        fig = px.scatter(df3, x='Payload Mass (kg)', y= 'class', color = 'Booster Version',
                        title='Payload and launchsite scatterpot')
        return fig


# Run the app
if __name__ == '__main__':
    app.run_server()


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [31/May/2022 15:55:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2022 15:55:17] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2022 15:55:17] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2022 15:55:17] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2022 15:55:17] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2022 15:55:17] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2022 15:55:17] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2022 15:55:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2022 15:55:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2022 15:55:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2022 15:55:45] "POST /_dash-